In [1]:
import pandas as pd

# Load the datasets
customers_df = pd.read_csv('Customers.csv')
products_df = pd.read_csv('Products.csv')
transactions_df = pd.read_csv('Transactions.csv')

# Check the first few rows of each dataframe to understand the data structure
print(customers_df.head())
print(products_df.head())
print(transactions_df.head())


  CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  2022-07-10
1      C0002      Elizabeth Lutz           Asia  2022-02-13
2      C0003      Michael Rivera  South America  2024-03-07
3      C0004  Kathleen Rodriguez  South America  2022-10-09
4      C0005         Laura Weber           Asia  2022-08-15
  ProductID              ProductName     Category   Price
0      P001     ActiveWear Biography        Books  169.30
1      P002    ActiveWear Smartwatch  Electronics  346.30
2      P003  ComfortLiving Biography        Books   44.12
3      P004            BookWorld Rug   Home Decor   95.69
4      P005          TechPro T-Shirt     Clothing  429.31
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3       

In [2]:
# Merge dataframes
data = pd.merge(transactions_df, customers_df, on="CustomerID", how="left")
data = pd.merge(data, products_df, on="ProductID", how="left")

# Check the merged data
print(data.head())


  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x     CustomerName         Region  SignupDate  \
0      300.68   300.68   Andrea Jenkins         Europe  2022-12-03   
1      300.68   300.68  Brittany Harvey           Asia  2024-09-04   
2      300.68   300.68  Kathryn Stevens         Europe  2024-04-04   
3      601.36   300.68  Travis Campbell  South America  2024-04-11   
4      902.04   300.68    Timothy Perez         Europe  2022-03-15   

                       ProductName     Category  Price_y  
0  ComfortLiving Bluetooth Speaker  Electronics   300.68  
1  ComfortLiving Bluetooth Speaker

In [3]:
# Aggregate data by CustomerID (summarize transaction data)
customer_transactions = data.groupby('CustomerID').agg({
    'TotalValue': 'sum',               # Total spending by customer
    'Quantity': 'sum',                 # Total quantity of products purchased
    'ProductID': pd.Series.nunique,    # Number of different products purchased
    'ProductName': pd.Series.nunique   # Number of unique products purchased
}).reset_index()

# Check the aggregated data
print(customer_transactions.head())


  CustomerID  TotalValue  Quantity  ProductID  ProductName
0      C0001     3354.52        12          5            5
1      C0002     1862.74        10          4            4
2      C0003     2725.38        14          4            4
3      C0004     5354.88        23          8            8
4      C0005     2034.24         7          3            3


In [9]:
from sklearn.preprocessing import OneHotEncoder

# One-Hot Encoding for the 'Region' column in customers_df
encoder = OneHotEncoder(sparse_output=False)  # Use sparse_output instead of sparse
region_encoded = encoder.fit_transform(customers_df[['Region']])

# Create a dataframe for the encoded region features
region_df = pd.DataFrame(region_encoded, columns=encoder.get_feature_names_out())
customers_df = pd.concat([customers_df, region_df], axis=1)

# Check the result of encoding
print(customers_df.head())


  CustomerID        CustomerName         Region  SignupDate  Region_Asia  \
0      C0001    Lawrence Carroll  South America  2022-07-10          0.0   
1      C0002      Elizabeth Lutz           Asia  2022-02-13          1.0   
2      C0003      Michael Rivera  South America  2024-03-07          0.0   
3      C0004  Kathleen Rodriguez  South America  2022-10-09          0.0   
4      C0005         Laura Weber           Asia  2022-08-15          1.0   

   Region_Europe  Region_North America  Region_South America  
0            0.0                   0.0                   1.0  
1            0.0                   0.0                   0.0  
2            0.0                   0.0                   1.0  
3            0.0                   0.0                   1.0  
4            0.0                   0.0                   0.0  


In [5]:
from sklearn.preprocessing import StandardScaler

# Normalize numerical features like TotalValue and Quantity
scaler = StandardScaler()
customer_transactions[['TotalValue', 'Quantity']] = scaler.fit_transform(
    customer_transactions[['TotalValue', 'Quantity']]
)

# Check the scaled features
print(customer_transactions.head())


  CustomerID  TotalValue  Quantity  ProductID  ProductName
0      C0001   -0.061701 -0.122033          5            5
1      C0002   -0.877744 -0.448000          4            4
2      C0003   -0.405857  0.203934          4            4
3      C0004    1.032547  1.670787          8            8
4      C0005   -0.783929 -0.936951          3            3


In [12]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming customers_df and products_df are already loaded

# Perform One-Hot Encoding for categorical variables like Region and Category
encoder = OneHotEncoder(sparse_output=False)

# One-Hot encode 'Region' (from customers_df) and 'Category' (from products_df)
region_encoded = encoder.fit_transform(customers_df[['Region']])
category_encoded = encoder.fit_transform(products_df[['Category']])

# Convert these encoded arrays back to dataframes
region_df = pd.DataFrame(region_encoded, columns=encoder.get_feature_names_out())  # Removed the input feature names
category_df = pd.DataFrame(category_encoded, columns=encoder.get_feature_names_out())  # Removed the input feature names

# Merge these encoded columns into their respective dataframes
customers_df = pd.concat([customers_df, region_df], axis=1)
products_df = pd.concat([products_df, category_df], axis=1)

# Now merge all dataframes together (transactions, customer, product data)
data = pd.merge(transactions_df, customers_df, on="CustomerID", how="left")
data = pd.merge(data, products_df, on="ProductID", how="left")

# Aggregate the data to get customer-level information (total spending, product counts, etc.)
customer_transactions = data.groupby('CustomerID').agg({
    'TotalValue': 'sum', 
    'Quantity': 'sum',
    'ProductID': pd.Series.nunique,
}).reset_index()

# Merge back the customer and product features for similarity computation
features = customer_transactions.drop(['CustomerID'], axis=1)  # Drop non-numeric columns
features = pd.concat([features, region_df, category_df], axis=1)  # Add encoded categorical features

# Handle missing values (you can choose one of these methods)
features = features.fillna(0)  # Or use features.fillna(features.mean()) or features.dropna()

# Standardize numerical features
scaler = StandardScaler()
features[['TotalValue', 'Quantity']] = scaler.fit_transform(features[['TotalValue', 'Quantity']])

# Now calculate cosine similarity
similarity_matrix = cosine_similarity(features)

# Check the shape of the similarity matrix
print(similarity_matrix.shape)


(200, 200)


In [13]:
# Assuming the previous steps are correct

# Make sure the 'features' DataFrame doesn't contain NaN values before computing similarity
features = features.fillna(0)  # or fillna(features.mean()) if you want to fill with the mean

# Standardize the numerical features again if needed
scaler = StandardScaler()
features[['TotalValue', 'Quantity']] = scaler.fit_transform(features[['TotalValue', 'Quantity']])

# Now calculate cosine similarity
similarity_matrix = cosine_similarity(features)

# Ensure similarity_matrix is correctly calculated and available
print(similarity_matrix.shape)  # Check if the matrix was generated

# Assuming get_top_3_lookalikes is defined, call it for the first 20 customers
lookalikes = []
for customer_id in customers_df['CustomerID'].iloc[:20]:
    lookalikes.extend(get_top_3_lookalikes(customer_id, similarity_matrix))

# Check the results for the first customer
print(lookalikes[0])


(200, 200)
('C0001', ['C0107', 'C0003', 'C0048'], [0.9809128736900199, 0.9920189297704356, 0.9986540040379928])


In [14]:
def get_top_3_lookalikes(customer_id, similarity_matrix):
    # Find the index of the customer in the data
    customer_index = customers_df[customers_df['CustomerID'] == customer_id].index[0]
    
    # Get the similarity scores for the customer
    similarity_scores = similarity_matrix[customer_index]
    
    # Get the indices of the top 3 most similar customers
    top_3_indices = similarity_scores.argsort()[-4:-1][::-1]  # Exclude the customer itself (index 0)
    
    # Retrieve the CustomerIDs of the most similar customers
    top_3_customers = customers_df.iloc[top_3_indices]['CustomerID'].values
    return top_3_customers


In [15]:
# Convert the results into a DataFrame
lookalike_df = pd.DataFrame(lookalikes, columns=['CustomerID', 'Lookalikes', 'Scores'])

# Save to CSV
lookalike_df.to_csv('Lookalike.csv', index=False)

# Check the output
print(lookalike_df.head())


  CustomerID             Lookalikes  \
0      C0001  [C0107, C0003, C0048]   
1      C0002  [C0056, C0027, C0088]   
2      C0003  [C0174, C0048, C0001]   
3      C0004  [C0102, C0169, C0113]   
4      C0005  [C0146, C0159, C0092]   

                                              Scores  
0  [0.9809128736900199, 0.9920189297704356, 0.998...  
1  [0.9801086612191465, 0.9934624308952046, 0.995...  
2  [0.9678396355982816, 0.9864088743333652, 0.992...  
3  [0.9897757651034574, 0.9915044528991657, 0.992...  
4  [0.9451792332944101, 0.9579115659581428, 0.979...  
